In [1]:
import pandas as pd
import numpy as np
name = "billing_and_accruals_for_script_2024-09-02_06-25-18"

In [2]:
dtype_dict = {
    'КПП': str,
}

In [3]:
dataset = pd.read_excel('C:/Users/Boris/Downloads/' + name + '.xlsx', dtype=dtype_dict)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9468 entries, 0 to 9467
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Название компании           9391 non-null   object        
 1   Дата                        9468 non-null   datetime64[ns]
 2   Личный кабинет              9468 non-null   int64         
 3   ЛС                          9303 non-null   object        
 4   ИНН                         8437 non-null   object        
 5   КПП                         5427 non-null   object        
 6   Номер договора              9303 non-null   object        
 7   Идентификатор номенклатуры  9303 non-null   float64       
 8   Название номенклатуры       9303 non-null   object        
 9   Стоимость                   9468 non-null   float64       
 10  Сумма                       9468 non-null   float64       
 11  Начисления partner          4374 non-null   float64     

In [5]:
dataset['КПП'] = dataset['КПП'].astype(str)

In [6]:
dataset.head(2)

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,NaN
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,NaN


In [7]:
dataset['Начисления partner'] = dataset['Начисления partner'].fillna(0)

In [8]:
dataset['Сумма-test'] = dataset['Сумма']

In [9]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,0.0,290.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,0.0,230.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30 03:00:00,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9463,Цифрал Тольятти,2024-07-31 03:00:00,152762,Подписка ПРО ЛК 152762,NaN,nan,Подписка ПРО ЛК 152762,40.0,Подписка ПРО,2.7,65458.8,0.0,65458.8
9464,Цифрал Тольятти,2024-08-31 03:00:00,152762,152762,NaN,nan,152762,50.0,Архивация видеопотока с заданной глубиной хран...,2.8,220606.4,0.0,220606.4
9465,Цифрал Тольятти,2024-08-31 03:00:00,152762,Подписка ПРО ЛК 152762,NaN,nan,Подписка ПРО ЛК 152762,40.0,Подписка ПРО,2.7,184771.8,0.0,184771.8
9466,Цифрал Тольятти,2024-09-30 03:00:00,152762,152762,NaN,nan,152762,50.0,Архивация видеопотока с заданной глубиной хран...,2.8,14252.0,0.0,14252.0


In [10]:
for i in range(0,dataset.shape[0]):
    if i == 0:
        date = dataset.loc[0,'Дата']
        lk = dataset.loc[0,'Личный кабинет'] 
        dataset.loc[0,'ballance'] = 0
        dataset.loc[0,'ballance'] = dataset.loc[0,'Сумма'] - dataset.loc[0, 'Начисления partner']
        if dataset.loc[0,'ballance'] < 0:
            dataset.loc[0,'Сумма']  = 0
        if dataset.loc[0,'ballance'] >= 0:
                dataset.loc[0,'Сумма'] = dataset.loc[0,'ballance'] 
                dataset.loc[0,'ballance'] = 0
            
    else:
    
        if date == dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if date != dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] - dataset.loc[i, 'Начисления partner'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if lk != dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            lk = dataset.loc[i,'Личный кабинет']
            
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] \
                                        - dataset.loc[i, 'Начисления partner']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0



In [11]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test,ballance
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,0.0,290.0,0.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0,0.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0,0.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,0.0,230.0,0.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30 03:00:00,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9463,Цифрал Тольятти,2024-07-31 03:00:00,152762,Подписка ПРО ЛК 152762,NaN,nan,Подписка ПРО ЛК 152762,40.0,Подписка ПРО,2.7,65458.8,0.0,65458.8,0.0
9464,Цифрал Тольятти,2024-08-31 03:00:00,152762,152762,NaN,nan,152762,50.0,Архивация видеопотока с заданной глубиной хран...,2.8,220606.4,0.0,220606.4,0.0
9465,Цифрал Тольятти,2024-08-31 03:00:00,152762,Подписка ПРО ЛК 152762,NaN,nan,Подписка ПРО ЛК 152762,40.0,Подписка ПРО,2.7,184771.8,0.0,184771.8,0.0
9466,Цифрал Тольятти,2024-09-30 03:00:00,152762,152762,NaN,nan,152762,50.0,Архивация видеопотока с заданной глубиной хран...,2.8,14252.0,0.0,14252.0,0.0


In [12]:
dataset.to_excel('C:/Users/Boris/Downloads/'+ name + '_test.xlsx', index=False)

In [13]:
dataset_ready = dataset.drop(['Начисления partner','Сумма-test','ballance'], axis= 1)

In [14]:
dataset_ready['Количество'] = round(dataset_ready['Сумма']/dataset_ready['Стоимость'])
dataset_ready['Количество'] = dataset_ready['Количество'].fillna(0) 

In [15]:
dataset_ready

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Количество
0,"ООО ""Видеоглаз Вэст""",2022-03-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,29.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,30.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,30.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30 03:00:00,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,23.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30 03:00:00,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9463,Цифрал Тольятти,2024-07-31 03:00:00,152762,Подписка ПРО ЛК 152762,NaN,nan,Подписка ПРО ЛК 152762,40.0,Подписка ПРО,2.7,65458.8,24244.0
9464,Цифрал Тольятти,2024-08-31 03:00:00,152762,152762,NaN,nan,152762,50.0,Архивация видеопотока с заданной глубиной хран...,2.8,220606.4,78788.0
9465,Цифрал Тольятти,2024-08-31 03:00:00,152762,Подписка ПРО ЛК 152762,NaN,nan,Подписка ПРО ЛК 152762,40.0,Подписка ПРО,2.7,184771.8,68434.0
9466,Цифрал Тольятти,2024-09-30 03:00:00,152762,152762,NaN,nan,152762,50.0,Архивация видеопотока с заданной глубиной хран...,2.8,14252.0,5090.0


In [16]:
dataset_ready.to_excel('C:/Users/Boris/Downloads/'+ name + '_ready.xlsx', index=False)